This notebook is a prototype of the rTMS Bayesian Optimization algorithm and will reflect the final step of pipeline after tetrahedral projection and FEM surface construction/registration

In [1]:
import os
import numpy as np
from fieldopt import geolib
from fieldopt.objective import FieldFunc
from timeit import default_timer as timer

ImportError: No module named fieldopt

In [2]:
#MOE package loading (should wrap!)
from moe.optimal_learning.python.cpp_wrappers.domain import TensorProductDomain as cTensorProductDomain
from moe.optimal_learning.python.python_version.domain import TensorProductDomain
from moe.optimal_learning.python.geometry_utils import ClosedInterval
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import ExpectedImprovement
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import multistart_expected_improvement_optimization as meio
from moe.optimal_learning.python.data_containers import HistoricalData, SamplePoint
from moe.optimal_learning.python.cpp_wrappers.log_likelihood_mcmc import GaussianProcessLogLikelihoodMCMC
from moe.optimal_learning.python.default_priors import DefaultPrior
from moe.optimal_learning.python.python_version.optimization import GradientDescentOptimizer, GradientDescentParameters
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentOptimizer as cGDOpt
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentParameters as cGDParams

In [ ]:
import objgraph
import gc

In [ ]:
#Specify inputs
proj_dir =  '/projects/jjeyachandra/rtms_optimize'
mesh_file = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')
coil_file = os.path.join(proj_dir,'resources','coils','Magstim_70mm_Fig8.nii.gz')
tet_file = os.path.join(proj_dir,'output','tetra_parcels')
C_file = os.path.join(proj_dir,'output','quadratic_vec')
iR_file = os.path.join(proj_dir,'output','inverse_rot')
b_file = os.path.join(proj_dir,'output','param_bounds')
testing_dir = '/tmp/'#os.path.join(proj_dir,'testing','tmp')

In [ ]:
#Step 1: Load in files containing relevant information about search domain
C = np.fromfile(C_file)
iR = np.fromfile(iR_file).reshape(3,3)
b = np.fromfile(b_file).reshape(3,2)

#Load in element mask
p_map = np.load(os.path.join(proj_dir,'output','tetra_parcels.npy'))[:,2]

In [ ]:
#Step 2: Specify the search domain
search_domain = TensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), #X coord on quadratic surface
    ClosedInterval(b[1,0],b[1,1]), #Y coord on quadratic surface
    ClosedInterval(0,180) #Rotational angle
])

c_search_domain = cTensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), 
    ClosedInterval(b[1,0],b[1,1]),
    ClosedInterval(0,180)
])

In [ ]:
#Make objective function
f = FieldFunc(mesh_file=mesh_file, quad_surf_consts=C,
              surf_to_mesh_matrix=iR, tet_weights=p_map,
              field_dir=testing_dir, coil=coil_file, cpus=6)

In [ ]:
#Generate historical data 
hist_pts = 6
init_pts = search_domain.generate_uniform_random_points_in_domain(hist_pts)
start = timer()
observations = f.evaluate(init_pts)
stop = timer()
print(stop - start)

In [ ]:
TMSLIST = obj[0]

In [ ]:
objgraph.show_backrefs(obj[0],max_depth=3,filename='./cond.png')

In [ ]:
observations = -observations

In [ ]:
hist_data = HistoricalData(dim = 3, num_derivatives= 0)
hist_data.append_sample_points([SamplePoint(inp,o,0.0) 
                                for o,inp in 
                                zip(observations,init_pts)])


In [ ]:
#Set up model specifications
prior = DefaultPrior(n_dims = 3 + 2, num_noise=1)
gp_ll = GaussianProcessLogLikelihoodMCMC(historical_data=hist_data,
                                         derivatives=[], prior=prior,
                                         chain_length=1000, burnin_steps=2000,
                                         n_hypers=2**4, noisy=False)
gp_ll.train()

In [ ]:
#Set up optimizer
sgd_params = cGDParams(num_multistarts=200, max_num_steps=50,
                      max_num_restarts=2, num_steps_averaged=4,
                      gamma=0.7, pre_mult=1.0, max_relative_change=0.5,
                      tolerance=1.0e-10)

In [ ]:
#Set up optimization parametes
from IPython.display import clear_output, display
num_iters = 50

In [ ]:
def gen_sample_from_qei(gp,search_domain,sgd_params,num_samples, num_mc=1e4, lhc_iter=2e4):
    
    qEI = ExpectedImprovement(gaussian_process=gp, num_mc_iterations=int(num_mc))
    optimizer = cGDOpt(search_domain, qEI, sgd_params, int(lhc_iter))
    points_to_sample = meio(optimizer, None, num_samples, use_gpu=False, which_gpu=0,
                      max_num_threads=8)
    qEI.set_current_point(points_to_sample[0])
    
    return points_to_sample, qEI.compute_expected_improvement()

In [ ]:
best_point_history = []
for i in np.arange(0,num_iters):
    
    #Optimize qEI and pick samples
    points_to_sample, ei = gen_sample_from_qei(gp_ll.models[0],
                                               c_search_domain, sgd_params=sgd_params,
                                               num_samples=6, num_mc=2**10)
    
    #Collect observations
    sampled_points = -f.evaluate(points_to_sample)
    evidence = [SamplePoint(c,v,0.0) for c,v in zip(points_to_sample, sampled_points)]
    
    #Update model
    gp_ll.add_sampled_points(evidence)
    gp_ll.train()
    
    #Pull model and pull values
    gp = gp_ll.models[0]
    min_point = np.argmin(gp._points_sampled_value)
    min_val = np.min(gp._points_sampled_value)
    best_coord = gp.get_historical_data_copy().points_sampled[min_point]
    
    clear_output()
    print('Recommended Points:')
    print(points_to_sample)
    print('Expected Improvement: {}'.format(ei))
    print('Current Best:')
    print('f(x*)=',min_val)
    print('Coord:', best_coord)
    
    best_point_history.append(min_val)